# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/reachy-mini',
 '/spaces',
 '/models',
 '/THUDM/GLM-4.1V-9B-Thinking',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/kyutai/tts-1.6b-en_fr',
 '/apple/DiffuCoder-7B-cpGRPO',
 '/HuggingFaceTB/SmolLM3-3B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/kontext-community/kontext-relight',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/ilcve21/Sparc3D',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/hackaprompt/Pliny_HackAPrompt_Dataset',
 '/datasets/marcelbinz/Psych-101',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/facebook/seamless-interaction',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/micr

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'company social media', 'url': 'https://twitter.com/huggingface'},
  {'type': 'company social media',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Reachy Mini: The Open Robot for AI Builders
Welcome Cohere on the Hub 🔥
Welcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
THUDM/GLM-4.1V-9B-Thinking
Updated


In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nReachy Mini: The Open Robot for AI Builders\nWelcome Cohere on the Hub 🔥\nWelcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTHUDM/GLM-4.1V-9B-Thinking\nUpdated\n1 day ago\n•\n15.3k\n•\n469\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n12 days ago\n•\n188k\n•\n1.48k\nkyutai/tts-1.6b-en_fr\nUpdated\n1 day ago\n•\n15.9k\n•\n259\napple/DiffuCoder-7B-cpGRPO\nUpdated\n5 days 

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

---

## Welcome to Hugging Face – The AI Community Building the Future

At Hugging Face, we are pioneering a collaborative platform where the machine learning community converges to create, enhance, and share models, datasets, and applications. Our mission is to accelerate and democratize AI development for everyone.

### What We Offer

- **Innovative Models**: Access over 1 million machine learning models, including state-of-the-art natural language processing, computer vision, and audio processing tools.
  
- **Comprehensive Datasets**: Collaborate on and share from a vast array of over 250,000 datasets catering to diverse ML tasks.

- **Robust Spaces**: Create applications and interactive demos using our Spaces feature, which supports multiple modalities including text, image, and video.

- **Enterprise Solutions**: We empower organizations with advanced AI tools, enterprise-grade security, access controls, and dedicated support for team-based AI projects.

### Community and Collaboration

Join a vibrant community with over 50,000 organizations leveraging our platform, including prominent companies such as Meta, Google, Microsoft, and Amazon. Our commitment to open-source initiatives emphasizes collaboration and shared knowledge, as seen in projects like Transformers and Diffusers.

### Company Culture

At Hugging Face, we believe in fostering a supportive and inclusive company culture where innovation thrives. Our team is composed of passionate individuals committed to redefining the boundaries of AI. We encourage creativity, collaboration, and growth, ensuring that every voice is heard and valued. 

### Career Opportunities

As we continue to grow, we are on the lookout for talented individuals who share our passion for AI. We offer a variety of roles spanning from engineering and product management to community relations and marketing. Join us in building the future of AI!

- **Why Work with Us?**
  - A collaborative and inclusive work environment.
  - Opportunities to work on industry-leading projects.
  - Competitive salaries and benefits.

### Get Started

Explore the possibilities that Hugging Face has to offer. Whether you are an investor seeking to back an innovative tech company, a prospective recruit eager to join a forward-thinking team, or a customer ready to leverage our advanced AI capabilities, we welcome you to become part of our journey.

**Join us today and help shape the future of AI!**

[Visit our Website](https://huggingface.co) | [Join the Community](https://huggingface.co/join)

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

---

## About Us
Hugging Face is a pioneering platform dedicated to building the future of artificial intelligence. We serve as a vital community for machine learning enthusiasts, developers, and organizations, facilitating collaboration on innovative models, datasets, and applications. Our commitment to open-source technology drives our mission to democratize machine learning and foster a creative ecosystem where AI solutions can thrive.

## Our Offerings
- **Models**: Browse our extensive library of over 1 million models, encompassing state-of-the-art architectures in natural language processing, computer vision, and more.
- **Datasets**: Explore a vast collection of datasets—over 250,000 ready for use in a multitude of ML tasks.
- **Spaces**: Create and collaborate on applications with ease. Our dynamic platform enables users to host and share public models, datasets, and applications without limits.
- **Enterprise Solutions**: Our advanced enterprise services provide organizations with secure access controls, dedicated support, and optimized compute solutions tailored for team growth and efficiency.

## Customers
Hugging Face is trusted by leading organizations in diverse industries, including:
- Google
- Microsoft
- Amazon
- Grammarly
- Meta

With over 50,000 organizations utilizing our services, our platform is a cornerstone for AI and machine learning development.

## Company Culture
At Hugging Face, we believe in fostering a collaborative and inclusive environment. Our employees are part of a vibrant community that encourages innovation and creativity. We prioritize open communication, mutual support, and continuous learning, empowering each team member to contribute to our vision of advancing AI.

## Careers
Join us on our journey to revolutionize AI! We are constantly looking for passionate individuals to join our team. Visit our careers page to explore current openings and become an integral part of our mission. With opportunities ranging from technical roles to community engagement, there’s a place for everyone at Hugging Face.

## Join the Community
Become a part of the Hugging Face community today! Sign up to start exploring our extensive resources, connect with fellow AI enthusiasts, and contribute to the future of machine learning.

---

For more information, visit our website at [huggingface.co](https://huggingface.co) or follow us on social media to stay up-to-date with our latest innovations and community events!

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure

## Company Overview
**Hugging Face** is an innovative platform and community at the forefront of artificial intelligence (AI) and machine learning (ML). Focused on collaboration, Hugging Face provides a space where developers, researchers, and enthusiasts can come together to build the future of AI. From impressive models and comprehensive datasets to a thriving ecosystem of applications, Hugging Face is redefining the landscape of machine learning.

## Our Community
At Hugging Face, we embrace the power of community. With over 50,000 organizations using our platform, including industry giants like Google, Microsoft, and Amazon, we help foster collaboration across sectors. Our open-source ethos encourages contributions, making it easy for users to create, share, and discover quality ML resources. 

## Products and Services
- **Models**: Explore over **1 million models**, tailored for various AI and ML applications.
- **Datasets**: Access **250k+ datasets**, essential for training and evaluating models.
- **Spaces**: Run and implement applications efficiently through **400k+ applications.**
- **Enterprise Solutions**: Our platform provides enterprise-grade security and dedicated support for teams looking to leverage cutting-edge AI technology.

### Key Offerings
- **Compute Solutions**: Starting at **$0.60/hour for GPU**, we provide optimized inference endpoints and dedicated spaces for applications.
- **Team & Enterprise Solutions**: Designed for $20/user/month, offering features like Single Sign-On and priority support tailored for larger organizations.

## Company Culture
At Hugging Face, we emphasize **creativity, collaboration,** and a **supportive environment**. Our team of over 200 is dedicated to cultivating a culture centered around **innovation** and **community involvement**. We welcome diverse perspectives to refine and enhance our offerings.

### Join Our Team
As a rapidly growing organization, Hugging Face is continually seeking talent across various domains. Whether in AI research, engineering, or community outreach, we provide an exciting environment for professionals looking to make a mark in the AI landscape. 

Explore our **[Career Opportunities](https://huggingface.co/jobs)** to find out how you can contribute to building the future of AI.

## Be Part of the Future
Join us in the movement to build groundbreaking AI solutions and become part of a thriving community passionate about technology and innovation. **Sign up today** and start exploring a world of possibilities with Hugging Face!

### Connect with Us
- **Website**: [huggingface.co](https://huggingface.co)
- **GitHub**: [Hugging Face on GitHub](https://github.com/huggingface)
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **LinkedIn**: [Hugging Face](https://linkedin.com/company/huggingface)  
- **Discord**: Join our community discussions!

--- 
Embrace the power of AI with Hugging Face, where technology meets community.

In [23]:
#kendim denedim
import os
from IPython.display import Markdown, display, update_display

def custom_brochure(company_name, url, stream=False, save_to_file=None):
    """
    Şirket broşürü oluşturur ve ekranda gösterir ya da dosyaya kaydeder.
    :param company_name: Şirket adı (örn. "HuggingFace")
    :param url: Web sitesi URL'si (örn. "https://huggingface.co")
    :param stream: True ise akış halinde, False ise toplu yanıt
    :param save_to_file: Dosya yolu, broşürü kaydetmek için (örn. "brochure.md")
    """
    try:
        # Kullanıcı mesajını al
        user_prompt = get_brochure_user_prompt(company_name, url)
        
        # OpenAI isteği
        if stream:
            # Akış modunda
            stream_response = openai.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                stream=True
            )
            response = ""
            display_handle = display(Markdown(""), display_id=True)
            for chunk in stream_response:
                chunk_content = chunk.choices[0].delta.content or ''
                response += chunk_content
                response = response.replace("```", "").replace("markdown", "")
                update_display(Markdown(response), display_id=display_handle.display_id)
        else:
            # Toplu yanıt modunda
            response = openai.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            ).choices[0].message.content
            display(Markdown(response))

        # Dosyaya kaydetme (isteğe bağlı)
        if save_to_file:
            with open(save_to_file, 'w', encoding='utf-8') as f:
                f.write(response)
            print(f"Broşür kaydedildi: {save_to_file}")

        return response

    except Exception as e:
        print(f"Hata oluştu: {str(e)}")
        return None

In [24]:
# Toplu yanıt, dosyaya kaydetmeden
custom_brochure("HuggingFace", "https://huggingface.co", stream=False)

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## About Us

**Hugging Face** is at the forefront of the machine learning community, dedicated to building a collaborative platform that empowers developers, researchers, and AI enthusiasts. Our mission is to build the future of artificial intelligence through a focus on community-driven innovation. With an extensive repository of over 1 million models and 250k datasets, Hugging Face is the home of machine learning, enabling users to discover, create, and share their work seamlessly.

### Our Community

At Hugging Face, we believe in the power of collaboration. We serve as the platform where the machine learning community can work together to develop applications, experiment with datasets, and share their breakthroughs. With over 50,000 organizations—including giants like Google, Amazon, Microsoft, and Meta—using our tools, our community is robust and ever-evolving.

### Our Offerings
- **Models:** Access a comprehensive library of machine learning models across various modalities: text, image, video, audio, and 3D.
- **Datasets:** Browse through thousands of datasets tailored for different ML tasks, ensuring easy access to quality data for your projects.
- **Spaces:** Build and share applications quickly and efficiently. Hugging Face hosts numerous applications ready for immediate use.
- **Enterprise Solutions:** Our advanced platform offers enterprise-grade security, access controls, and dedicated support for teams seeking to harness the power of AI in their operations.

## Company Culture

**Hugging Face** recognizes that our greatest strength lies in our people. We foster an inclusive, innovative, and collaborative environment where employees are encouraged to experiment, share ideas, and grow. Our commitment to open-source principles ensures that everyone has the opportunity to contribute to, and learn from, our platform.

- *Creative Freedom*: We empower our team members to explore and innovate, contributing to the broader Machine Learning ecosystem.
- *Diversity and Inclusion*: We value diversity in perspectives and experiences, ensuring a welcoming environment for people from all backgrounds.

## Careers at Hugging Face

Join us in building the future of AI! We are constantly on the lookout for talented individuals who share our passion for machine learning and innovation. We offer a wide range of positions in various fields, including:

- Software Development
- Machine Learning Research
- Data Science
- Community Engagement

Explore our current job openings and become a part of our growing team at [Hugging Face Careers](#).

## Join Us

We invite you to explore the world of AI with us. Whether you’re a customer, investor, or potential recruit, the possibilities are endless at Hugging Face. Together, let’s shape the future of artificial intelligence!

For more information, visit our website:
[www.huggingface.co](https://huggingface.co)

---

*Hugging Face – The AI community building the future.*

'# Hugging Face Company Brochure\n\n## About Us\n\n**Hugging Face** is at the forefront of the machine learning community, dedicated to building a collaborative platform that empowers developers, researchers, and AI enthusiasts. Our mission is to build the future of artificial intelligence through a focus on community-driven innovation. With an extensive repository of over 1 million models and 250k datasets, Hugging Face is the home of machine learning, enabling users to discover, create, and share their work seamlessly.\n\n### Our Community\n\nAt Hugging Face, we believe in the power of collaboration. We serve as the platform where the machine learning community can work together to develop applications, experiment with datasets, and share their breakthroughs. With over 50,000 organizations—including giants like Google, Amazon, Microsoft, and Meta—using our tools, our community is robust and ever-evolving.\n\n### Our Offerings\n- **Models:** Access a comprehensive library of machine l

In [25]:
stream_brochure("Anthropic", "https://www.anthropic.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}]}


# Anthropic Brochure

## Who We Are

At **Anthropic**, we are at the forefront of AI development, committed to creating and advancing artificial intelligence technologies that prioritize safety and human benefit. Our flagship AI model, **Claude**, encompasses a sophisticated range of capabilities tailored for diverse applications including coding, customer support, and education. 

## Our Products

### Claude Models
- **Claude Opus 4**: Our most intelligent AI model yet, designed for handling extensive tasks across various functions.
- **Claude Sonnet 4**: Blends power and versatility, ideal for coding and AI agent development.
- **Claude Haiku 3.5**: The foundational model that paved the way for our other offerings.

### Solutions We Offer
- **AI Agents**: Enhanced tools for automating and facilitating tasks in various industries.
- **Coding Assistance**: Helping developers optimize their workflows and code more efficiently.
- **Customer Support Solutions**: Creating better interactions between businesses and customers through intelligent communication tools.
- **Education Tools**: Leveraging AI for improved learning experiences in academic settings.

## Commitment to Safety

We recognize the profound implications of AI on society. At Anthropic, we are dedicated to responsible AI development, focusing on:
- **Transparency in our processes**.
- **Responsible Scaling Policies** that take into account the societal impacts of AI.
- **Active Research** on AI safety, ensuring our technologies serve humanity's long-term well-being.

## Company Culture

Our team thrives on collaboration, innovation, and a shared commitment to addressing the challenges that come with advancing AI technology. We foster an environment where ideas are freely shared and personal growth is encouraged. We believe in the importance of **learning and development**, exemplified by our **Anthropic Academy**, which equips individuals with the knowledge to build and innovate with Claude.

## Careers at Anthropic

We are continuously looking for passionate individuals who are eager to contribute to the future of safe AI. If you’re interested in shaping the way AI integrates into our lives, explore our open positions. Join us in building tools that prioritize human values and safety in technology.

- **Explore Opportunities**: Check our careers page for current job openings and events.

## Customer Focus

Our diverse client base ranges from startups to large enterprises, all leveraging Claude's capabilities to enhance their services and streamline operations. We prioritize understanding our customers' needs, striving toward solutions that empower them in their unique journeys.

---

Learn more about us and engage with our technology at [Anthropic.ai](https://www.anthropic.ai). Join us in building a future where AI serves humanity responsibly and effectively!

In [26]:
stream_brochure("VBAP", "https://www.vbap.com.tr/tr_TR/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.vbap.com.tr/tr_TR/kurumsal'}, {'type': 'homepage', 'url': 'https://www.vbap.com.tr/tr_TR'}, {'type': 'products page', 'url': 'https://www.vbap.com.tr/tr_TR/category/urunler'}, {'type': 'careers page', 'url': 'https://www.vbap.com.tr/tr_TR/brosurler'}]}


# VBAP Company Brochure

**Contact Us:**
- Phone: +90 224 441 46 44
- Email: info@vbap.com.tr

---

## About VBAP

Welcome to **VBAP**, a leading consulting firm specializing in **SAP**, **ERP**, software solutions, and training. Our mission is to empower businesses by enhancing operational efficiency and transparency through advanced technology. We aim to be a powerful global IT brand, addressing contemporary and future expectations in the IT sector with top-notch applications and consultancy services.

---

## Our Solutions

At VBAP, we offer comprehensive solutions tailored to meet the unique needs of your organization:

- **SAP Integrated Budget Module:** Simplify financial management and avoid undesirable surprises.
- **SAP Integrated ZMMC Module:** Enhance material management processes with unique SAP integration advantages.
- **SAP Integrated vSRM Solution:** Optimize supplier relations for sustainable growth.
- **Smart Warehouse Systems:** Digitally transform and optimize warehouse operations.
- **Professional Resource Management:** Streamline workflows for increased efficiency and transparency.

With our expertise in **SAP ECC**, **EDI**, **SAP Mobility**, and **SAP BI**, we ensure that our clients achieve operational excellence.

---

## Company Culture

At VBAP, we believe that a thriving workplace leads to exceptional results. Our culture is built on:

- **Innovation:** We constantly seek advanced technological solutions to stay ahead in the IT environment.
- **Collaboration:** We work closely with clients to ensure that our services fit seamlessly into their business processes.
- **Continuous Learning:** Through our **VBAP Academy**, we foster a culture of learning and growth, encouraging our employees to expand their skills.

---

## Our Clients

We are proud to have partnered with notable organizations across sectors, delivering tailored solutions that enhance their operational capabilities. Our collaboration with **weHIRE** further strengthens our ability to provide innovative HR solutions, enabling businesses to build efficient and agile teams.

---

## Careers at VBAP

Join a forward-thinking team dedicated to making a difference in the IT consultancy space! We are on the lookout for talented individuals who are passionate about technology and eager to contribute to our mission. When you work with us, you will have the opportunity to:

- Collaborate on challenging projects with industry experts.
- Benefit from a culture of continuous learning and professional development.
- Engage in innovative solutions that change the way businesses operate.

If you are interested in joining us, please reach out to us at info@vbap.com.tr.

---

**Choose VBAP - Where Your Business Gains Value!**  
Experience the next level of business management with our SAP ERP solutions. Let’s embark on this transformative journey together!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>